## Required Modules !

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,SnowballStemmer
from nltk.corpus import stopwords
import math

## Data - Data !

In [2]:
# Input-ting the X and Y datasets !

In [3]:
f = open("/home/khushvirmani/Desktop/wholeDesktop/movieReviewProject/imdbTrainDataX.txt", 'r')
lines = f.readlines()
f.close()

h = open("/home/khushvirmani/Desktop/wholeDesktop/movieReviewProject/imdbTrainDataY.txt", 'r')
ratings = h.read()
h.close()

In [4]:
rated = ratings.split("\n")
ratingsList = rated[:-1]
print(len(ratingsList))
print(len(lines))

25000
25000


## Cleaning Data !

In [6]:
# Tokenizing the data and then cleaning the data by StopWord removal and stemming !
def wordClean(string):
    # Tokenize
    tokenizer = RegexpTokenizer("[a-zA-Z']+")
    textPh1 = tokenizer.tokenize(string.lower())
    
    # Stopword removal
    stpwrd = stopwords.words('english')
    textPh2 = [ix for ix in textPh1 if ix not in stpwrd]
    
    # Stemming
    stemmer = SnowballStemmer('english') 
    textPh3 = [stemmer.stem(ix) for ix in textPh2]
    
    return textPh3

In [7]:
# clean Training data !
cleanReviews  = [ ]
for ix in lines:
    wd = wordClean(ix)
    cleanReviews.append(wd)

In [8]:
# training labels !
ratingsTrain = []
for jx in ratingsList:
    ratingsTrain.append(int(jx))

In [43]:
# Uncomment to check the data !!

# for ix in range(4):
#     print(cleanReviews[ix])
#     print(ratingsTrain[ix])
#     print("\n")

In [9]:
# print(len(cleanReviews))
# print(len(ratingsTrain))

25000
25000


## Creating workable Vocabulary !

In [11]:
# Count of each review label .

priorCount = { }
for ix in ratingsTrain:
    if priorCount.get(ix) == None:
        priorCount[ix] = 1
    else:
        priorCount[ix] += 1

In [12]:
# Prior Probability of each review label .

priorProb = { }
for ix in priorCount:
    priorProb[ix] = priorCount[ix]/float(len(ratingsTrain))

In [14]:
# total frequency of each word w.r.t. each class !

wordFreqPerCLass = { }
for ix in range(len(cleanReviews)):
    for jx in cleanReviews[ix]:
        if wordFreqPerCLass.get(jx) == None:
            wordFreqPerCLass[jx] = { }
        else:    
            if wordFreqPerCLass[jx].get(ratingsTrain[ix]) == None:
                wordFreqPerCLass[jx][ratingsTrain[ix]] = 1
            else:
                wordFreqPerCLass[jx][ratingsTrain[ix]] += 1 

In [16]:
# Global labels, as review categories 6,7 do not exist !

labelArgmax = {0:1,1:2,2:3,3:4,4:7,5:8,6:9,7:10}

In [13]:
# the total frequncy of each word in all reviews .
## NOT REQUIRED !!

# wordsFreq = { }
# for ix in cleanReviews:
#     for jx in ix:
#         if wordsFreq.get(jx) == None:
#             wordsFreq[jx] = 1
#         else:
#             wordsFreq[jx]+=1

In [15]:
# total words in each class !
## NOT Required !

# wordsInClasses = {1:0,2:0,3:0,4:0,7:0,8:0,9:0,10:0}
# for ix in range(len(cleanReviews)):
#     wordsInClasses[ratingsTrain[ix]]+= len(cleanReviews[ix])    

## tf-idf scores !

In [18]:
# def tfIdf(tf_word , total_docs, docs_present):
#     return tf * math.log2((total_docs+1)

## The MAIN code !

In [19]:
f = open('/home/khushvirmani/Desktop/wholeDesktop/movieReviewProject/imdbtestDataX.txt')
testReviews = f.read()
f.close()

In [20]:
g = open("/home/khushvirmani/Desktop/wholeDesktop/movieReviewProject/imdbtestDataY.txt")
testY = g.read()
g.close()

In [21]:
testReviews = testReviews.split("\n")
cleanTestReviews  = [ ]
for ix in testReviews:
    wdTest = wordClean(ix)
    cleanTestReviews.append(wdTest)
cleanTestReviews = cleanTestReviews[:25000]    

In [22]:
testLabels = testY.split("\n")
testLabels = testLabels[:25000]

In [45]:
# UNCOMMENT to check the test data !

# for ix in range(4):
#     print(cleanTestReviews[ix])
#     print(testLabels[ix])
#     print("\n")

In [24]:
predictedY = []
for ix in cleanTestReviews[:25000]:
    tempLst = []
    for kx in labelArgmax.values():
         # for hloding respective probabilities of classes !
        posteriorProbab = priorProb[kx]
        for jx in ix:
            if jx in wordFreqPerCLass.keys():
                if kx in wordFreqPerCLass[jx].keys():
                    posteriorProbab *= float(wordFreqPerCLass[jx][kx])/priorCount[kx]
                else:
                    posteriorProbab *= float(1/(priorCount[kx]))
            else:
                posteriorProbab *= float(1/(priorCount[kx]))
        tempLst.append(posteriorProbab)
#     print(tempLst)
    predictedY.append(labelArgmax[np.argmax(tempLst)])

In [49]:
cx = 0
for zx in range(len(predictedY)):
    if int(testLabels[zx]) == predictedY[zx]:
        cx+=1
print(cx)

4370


In [35]:
predictedTrainY = []
for ix in cleanReviews[:25000]:
    tempLst = []
    for kx in labelArgmax.values():
         # for hloding respective probabilities of classes !
        posteriorProbab = priorProb[kx]
        for jx in ix:
            if jx in wordFreqPerCLass.keys():
                if kx in wordFreqPerCLass[jx].keys():
                    posteriorProbab *= float(wordFreqPerCLass[jx][kx])/priorCount[kx]
                else:
                    posteriorProbab *= float(1/(priorCount[kx]))
            else:
                posteriorProbab *= float(1/(priorCount[kx]))
        tempLst.append(posteriorProbab)
#     print(tempLst)
    predictedTrainY.append(labelArgmax[np.argmax(tempLst)])

In [39]:
cz = 0
for zx in range(len(predictedTrainY)):
    if int(ratingsTrain[zx]) == predictedTrainY[zx]:
        cz+=1
print(cz)

8577


## Accuracy !

In [51]:
# Accuracy on train data !


print("Accuracy on train data: ",(float(cz)/ 25000)*100)

# Accuracy on test data !

print("Accuracy on train data: ",(float(cx)/25000) *100)

Accuracy on train data:  34.308
Accuracy on train data:  17.48
